# Load fit results from A1 and A2

In [1]:
using JLD
files = readdir("saves")

all_phi1_full_orig = Array(Any, 100)
all_phi1_full_log = Array(Any, 100)
all_phi1_red_orig = Array(Any, 100)
all_phi1_red_log = Array(Any, 100)

phi0_red_orig = []
phi0_red_log = []
phi0_full = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];

n = 100
for i = 1:n
    phi1_red_orig, phi1_red_log, fname = load("saves/figure_A2_hess_"*string(i)*".jld", "phi1_orig", "phi1_log", "A1_filename")
    phi1_full_orig, phi1_full_log = load("saves/"*fname, "phi1_orig", "phi1_log")
    
    if i == 1
        phi0_red_orig, phi0_red_log = load("saves/figure_A2_hess_"*string(i)*".jld", "phi0_orig", "phi0_log")
    end
        
    all_phi1_full_orig[i] = phi1_full_orig
    all_phi1_full_log[i] = phi1_full_log
    all_phi1_red_orig[i] = phi1_red_orig
    all_phi1_red_log[i] = phi1_red_log
end

In [2]:
all_phi1_full_orig = hcat(all_phi1_full_orig[1:n]...)
all_phi1_full_log = hcat(all_phi1_full_log[1:n]...)
all_phi1_red_orig = hcat(all_phi1_red_orig[1:n]...)
all_phi1_red_log = hcat(all_phi1_red_log[1:n]...)


6x100 Array{Float64,2}:
 3.20898e-8   1.35829e-8   2.46993e-8  …  1.55956e-7  1.84893e-7   5.50225e-9
 8.94143     12.5268      11.2877         5.94941     5.55527     16.0566    
 0.566465     0.517835     0.50775        0.553083    0.569307     0.498821  
 3.21388e-5   3.22868e-5   3.98173e-5     3.5816e-5   3.15686e-5   3.47983e-5
 8.95066      9.76507      4.22656        4.80449     5.05053      7.87794   
 1.12244      0.894068     2.33578     …  2.38582     2.15446      1.20695   

In [32]:
tmp = load("saves/figure_A1_hess_"*"8"*".jld") # Example with diverging parameters on fit
#tmp = load("saves/figure_A1_hess_"*"96"*".jld") # Example with diverging parameters on fit


Dict{ByteString,Any} with 8 entries:
  "ydata_log"  => [-12.3551,-13.9017,-10.9747,-9.70873,-8.05426,-7.0345,-7.3342…
  "hess_log"   => 8x8 Array{Float64,2}:…
  "phi1_orig"  => [7.693526035414763e-37,8.756576900815245e-33,0.06628129465823…
  "phi1_log"   => [3.6394852897319113e-7,3.120567823320987e-21,0.04126621255687…
  "D_log"      => [2755.9712101455953,779.9524874526993,144.10796909210856,17.8…
  "ydata_orig" => [1.32368e-6,2.1981e-6,1.02432e-5,6.3849e-5,0.000172283,0.0007…
  "hess_orig"  => 8x8 Array{Float64,2}:…
  "D_orig"     => [70.53801014987826,15.408704979783252,0.8272221937886602,0.14…

# Plot things here

using NBInclude
nbinclude("MBAM_plots.ipynb")
import MBAM_plots

include("../BK_functions/bk_setup_script.jl")


PlotlyJS.plot(MBAM_plots.plot(reshape(BK_simulator(phi0, hcat(x_grid...))[:], 8, 13)'))

PlotlyJS.plot(MBAM_plots.plot(reshape(tmp["ydata_orig"], 8, 13)'))

PlotlyJS.plot(MBAM_plots.plot(reshape(BK_simulator(tmp["phi1_orig"], hcat(x_grid...))[:], 8, 13)'))

[phi0 tmp["phi1_orig"]]

PlotlyJS.plot([MBAM_plots.plot(reshape(BK_simulator(phi0, hcat(x_grid...))[:], 8, 13)', mode="markers", color=0) 
                MBAM_plots.plot(reshape(BK_simulator(tmp["phi1_orig"], hcat(x_grid...))[:], 8, 13)', mode="lines")
    ])

# Generate CSV files for plotting elsewhere

In [99]:
# Output the simulated data point markers in a CSV format with: 
# first column - voltage, second column - Popen, third column error top, fourth column

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for original model
    tmp_vals = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp_vals.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp_vals[:], errbar[:], errbar[:]))
end

# Also output the example fit with just first col - voltage, second col - Popen
# but with voltage sampled more densely to make it a continous line

Vdense = minimum(V):0.001:maximum(V) # voltage sampled every 1 millivolt

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for the fit to noisy data
    tmp_vals = BK_simulator(tmp["phi1_orig"], hcat(x_grid_cur...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp_vals[:]))
end

0.0
0.7
4.0
12.0
22.0
55.0
70.0
95.0
0.0
0.7
4.0
12.0
22.0
55.0
70.0
95.0


In [42]:
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 tmp["phi1_orig"]])

In [100]:
# Output the statistics from all our data:
# For each model and assay
# and for each parameter 
# we will look at the standard deviation of the order of magnitude of fitted values: std(log10(fit_vals))

# First column: Original model Original Assay
# Second column: Reduced model Original Assay
# Third column: Original model Log Assay
# Fourth column: Reduced model Log Assay

writecsv("CSV/figure_A2_panel1_orig_orig.csv", std(log10(all_phi1_full_orig) ,2))
writecsv("CSV/figure_A2_panel1_red_orig.csv", std(log10(all_phi1_red_orig) ,2))
writecsv("CSV/figure_A2_panel1_orig_log.csv", std(log10(all_phi1_full_log) ,2))
writecsv("CSV/figure_A2_panel1_red_log.csv", std(log10(all_phi1_red_log) ,2))